<h1>[국민청원 데이터 분류하기]</h1>

<p>• 주목받을 만한 청원 분류하기</p>
<p>• '주목 받는다'는 것은 주관적인 일이기 때문에 주관적 판단을 배제하기 위해 딥러닝을 도입 </p>
<p>• 높은 청원 참여인원을 기록한 글들의 특징을 학습하여, 새로운 글이 입력되었을 때 청원 참여인원이 높은 글들과의 유사성을 계산하여 판단</p>


<h3>[크롤링]</h3>

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time 

result = pd.DataFrame()
for i in range(584274, 595226):
    URL = "http://www1.president.go.kr/petitions/"+str(i)
    
    response = requests.get(URL)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.find('h3', class_ = 'petitionsView_title')
    count = soup.find('span', class_ = 'counter')
    
    for content in soup.select('div.petitionsView_write > div.View_write'):
        content 
        a = []
        for tag in soup.select('ul.petitionsView_info_list > li'):
            a.append(tag.contents[1])
            
        if len(a) != 0:
            df1 = pd.DataFrame({'start' : [a[1]],
                                'end' : [a[2]],
                                'category' : [a[0]],
                                'count' : [count.text],
                                'title' : [title.text],
                                'content' : [content.text.strip()[0: 13000]]
                              
                               })
            result = pd.concat([result, df1])
            result.index = np.arange(len(result))
            
        if i % 60 == 0:
            print("Sleep 90seconds. Count:"+ str(i)
                +", Local Time:"
                + time.strftime('%Y-%m-%d', time.localtime(time.time()))
                +""+ time.strftime('%X', time.localtime(time.time()))
                +", Data Length: " + str(len(result)))
            time.sleep(90)

Sleep 90seconds. Count:584640, Local Time:2021-10-1821:52:08, Data Length: 24
Sleep 90seconds. Count:585360, Local Time:2021-10-1821:54:56, Data Length: 31
Sleep 90seconds. Count:590760, Local Time:2021-10-1822:06:56, Data Length: 93
Sleep 90seconds. Count:594240, Local Time:2021-10-1822:14:36, Data Length: 182


<h3>[크롤링 데이터 확인]</h3>

In [2]:
print(result.shape)

df = result
df.head()

(241, 6)


,start,end,category,count,title,content
0,2020-01-28,2020-02-27,기타,203,"보험회사로부터 사기당한 피해자로써, 보험회사 사기죄로 의율할수있도록 청원합니다","보험회사도, 보험가입자에게 법령을 위반하여..고객을 기망,착오케 하여, 지급하지않은..."
1,2020-01-28,2020-02-27,정치개혁,"2,433",현직 국회의원의 모든 선거 자료에 지난 국회 출석률을 표시하도록 해주세요.,긴 말 안 쓰겠습니다.\n\r\n현직 국회의원의 모든 선거 자료에 \r\n지난 국회...
2,2020-01-28,2020-02-27,안전/환경,171,인천시 송월재개발 빨리 진행되게 해주세요!!,인천시 중구 송월 주택 재개발구역에 살고 있는 주민입니다.\n\r\n이 지역이 10...
3,2020-01-28,2020-02-27,보건복지,"4,671",외국인 국민건강과 의료보험 폐지청원,우리나라 자국민 우선시되야할 국민건강 및 의료보험입니다\n\r\n그걸 외국인에게도 ...
4,2020-01-28,2020-02-27,행정,324,세무사 먹여살리는 복잡한 세금계산!!! 각종 세금 계산은 나라에서 해주세요!!,태어나서 처음으로 양도세를 내게 된 사람입니다.\r\n집안 대대로 있던 땅이 있었는...


<h4>[크롤링 직후의 데이터]</h4>

In [3]:
df.loc[1]

start                                              2020-01-28
end                                                2020-02-27
category                                                 정치개혁
count                                                   2,433
title               현직 국회의원의 모든 선거 자료에 지난 국회 출석률을 표시하도록 해주세요.
content     긴 말 안 쓰겠습니다.\n\r\n현직 국회의원의 모든 선거 자료에 \r\n지난 국회...
Name: 1, dtype: object

<h3>[전처리]</h3>

In [4]:
import re
def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]','',str(text))
    return text
def remove_special_char(text):
    text = re.sub('[^ㄱ-ㅣ가-힣 0-9]+','',str(text))
    return text
df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

<h4>[전처리되었는지 확인]</h4>

In [5]:
df.loc[1]['content']

'긴 말 안 쓰겠습니다현직 국회의원의 모든 선거 자료에 지난 국회 출석률을 표시하도록 의무화 해주세요'

<h2>[토크나이징]</h2>
<p>• 형태소 분석기 패키지인 Konlpy모듈 import</p>

In [6]:
from konlpy.tag import Okt
okt = Okt()

df['title_token'] = df.title.apply(okt.morphs)
df['content_token'] = df.content.apply(okt.nouns)

<h3>[파생변수 생성]</h3>

In [7]:
df['token_final'] = df.title_token + df.content_token

df['count'] = df['count'].replace({',' : ''}, regex = True).apply(lambda x : int(x))
print(df.dtypes)
df['label'] = df['count'].apply(lambda x: 'Yes' if x>=1000 else 'NO')

start            object
end              object
category         object
count             int64
title            object
content          object
title_token      object
content_token    object
token_final      object
dtype: object


<h4>[분석에 필요한 token_final과 label만 추출하여 df_drop에 저장]</h4>

In [8]:
df_drop = df[['token_final','label']]

<h3>[단어 임베딩을 위한 WordfVec import]</h3>
<p>• WordVec은 단어의 의미와 유사도를 반영하여 단어를 벡터로 표현하는 방식</p>

In [9]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(df_drop['token_final'],
                           sg = 1,
                           vector_size = 100,
                           window = 2,
                           min_count = 1,
                           workers = 4
                           )
print(embedding_model)

model_result = embedding_model.wv.most_similar("코로나")
print(model_result) 

Word2Vec(vocab=7535, vector_size=100, alpha=0.025)
[('번', 0.9965199828147888), ('발생', 0.9920846223831177), ('명', 0.9903942346572876), ('신천지', 0.9889833927154541), ('동선', 0.9887596368789673), ('감염', 0.9877769947052002), ('격리', 0.9877591729164124), ('전국', 0.9869346618652344), ('몇번', 0.9852475523948669), ('점점', 0.9833980798721313)]


<h3>[임베딩 모델 저장 및 로드]</h3>

In [10]:
from gensim.models import KeyedVectors

embedding_model.wv.save_word2vec_format('data\petitions_tokens_w2v')

loaded_model = KeyedVectors.load_word2vec_format
("data\petitions_tokens_w2v")
#model_result = loaded_model.most_similar("코로나")
print(model_result)

[('번', 0.9965199828147888), ('발생', 0.9920846223831177), ('명', 0.9903942346572876), ('신천지', 0.9889833927154541), ('동선', 0.9887596368789673), ('감염', 0.9877769947052002), ('격리', 0.9877591729164124), ('전국', 0.9869346618652344), ('몇번', 0.9852475523948669), ('점점', 0.9833980798721313)]


<h3>[데이터를 Train, Validation set으로 랜덤하게 분할후 분할결과 저장]</h3>

In [11]:
from numpy.random import RandomState

rng = RandomState()

tr = df_drop.sample(frac = 0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('data/train.csv', index=False, encoding='utf-8-sig')
val.to_csv('data/validation.csv', index=False, encoding='utf-8-sig')

<h3>[Field 클래스 정의]</h3>
<p>• 자연어 처리 라이브러리인 torchtext 임포트</p>

In [12]:
import torchtext
from torchtext.legacy.data import Field

def tokenizer(text):
    text = re.sub('[\[\]\']','',str(text))
    text = text.split(',')
    return text
TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential=False)

<h3>[데이터 불러오기]</h3>

In [13]:
from torchtext.legacy.data import TabularDataset

train, validation = TabularDataset.splits(
    path = 'data/',
    train = 'train.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = [('text', TEXT),('label', LABEL)],
    skip_header = True
)

print("Train:", train[0].text, train[0].label)
print("Validation:", validation[0].text,validation[0].label)

Train: ['자', ' 가격', ' 리자', ' 들', ' 도', ' 대학교', ' 시험', ' 을', ' 볼', ' 수', ' 있게', ' 해주세요', ' 현재', ' 수능', ' 시험', ' 안', ' 확', ' 진자', ' 반', ' 사람', ' 자가', ' 격리', ' 이로', ' 대학교', ' 시험', ' 볼', ' 수', ' 사람', ' 대학교', ' 시험', ' 보기', ' 위', ' 자격', ' 요건', ' 최저', ' 점수', ' 위해', ' 수능', ' 반', ' 확', ' 진자', ' 그', ' 사람', ' 최저', ' 점수', ' 대학교', ' 시험', ' 적성', ' 고사', ' 논술시험', ' 볼', ' 자격', ' 박탈', ' 대학교', ' 방법', ' 논술', ' 전형', ' 수능', ' 전형', ' 적성', ' 전형', ' 등', ' 수능시험', ' 뿐', ' 대학교', ' 시험', ' 자가', ' 격리', ' 응시', ' 수', ' 생각'] NO
Validation: ['대한민국', ' 해병대', ' 독립', ' 청원', ' 해병대', ' 창', ' 현재', ' 대한민국', ' 국가', ' 안보', ' 지대', ' 공헌', ' 전공', ' 유신', ' 독재', ' 정권', ' 시절', ' 국군', ' 조직', ' 법', ' 제', ' 군', ' 임무', ' 조항', ' 육해공군', ' 군', ' 해병대', ' 삭제', ' 수모', ' 이후', ' 해병대', ' 위상', ' 날', ' 갈수록', ' 약화', ' 실제', ' 북한', ' 연평도', ' 도발', ' 사건', ' 해병대', ' 변', ' 장비', ' 제때', ' 조달', ' 못', ' 뿐', ' 합참', ' 제', ' 목소리', ' 내지', ' 등', ' 차별', ' 대우', ' 온', ' 것', ' 현실', ' 임', ' 심지어', ' 해병대', ' 지원', ' 복무', ' 뒤', ' 전역', ' 역자', ' 국군', ' 조직', ' 법', ' 육해공군', ' 군

<h3>[단어장 및 DataLoader 정의]</h3>

In [14]:
import torch
from torchtext.vocab import Vectors
from torchtext.legacy.data import BucketIterator

vectors = Vectors(name="data/petitions_tokens_w2v")

TEXT.build_vocab(train,
                 vectors = vectors,min_freq = 1,max_size = None)
LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(
    datasets = (train,validation),
    batch_size = 8,
    device = device,
    sort = False
)
print('임베딩 벡터의 개수와 차원:{}'.format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원:torch.Size([6919, 100])


<h2>TextCNN 모델 설계</h2>
<p>• 문장을 단어 기준으로 나누어 컴퓨터가 단어를 이해할 수 있도록 텍스트를 벡터 형태로 전환하는 임베딩 실시</p>
<p>• TextCNN은 문장의 문맥적 의미를 파악하는 과정에서 정보를 집약해주기 때문에 연산이 빠르기때문에 사용</p>

In [15]:
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F

class TextCNN(nn.Module):
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)
        
    def forward(self, x):
        
            
        emb_x = self.embed(x)
        emb_x = emb_x.unsqueeze(1)
            
        con_x = [self.relu(conv(emb_x)) for conv in self.convs]
            
        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]
        fc_x = torch.cat(pool_x, dim=1)
        fc_x = fc_x.squeeze(-1)
        fc_x = self.dropout(fc_x)
        logit = self.fc(fc_x)
            
        return logit

<h3>[모델 학습 함수 정의]</h3>

In [16]:
def train(model, device, train_itr, optimizer):
    
    model.train()
    corrects, train_loss = 0.0,0
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        optimizer.zero_grad()
        logit = model(text)
        
        loss = F.cross_entropy(logit, target)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()
    
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)
    
    return train_loss, accuracy

<h3>[모델 평가 함수 정의]</h3>

In [17]:
def evaluate (model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0,0
    
    for batch in itr:
        
        text = batch.text
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)
        
        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()
        
    test_loss /= len(itr.dataset)
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

<h3>[모델 학습 및 성능 확인]</h3>

In [19]:
model = TextCNN(vocab, 100, 10,[3,4,5],2).to(device)
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1,3+1):
    
    tr_loss, tr_acc = train(model, device, train_iter, optimizer) 
    
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc = evaluate(model, device, validation_iter)
    
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))
    
    if val_acc > best_test_acc:
        best_test_acc = val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")
    
    print('-------------------------------------------------------------------------------')

TextCNN(
  (embed): Embedding(6919, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)
Train Epoch: 1 	 Loss: 0.08566091159464782 	 Accuracy: 63.73057174682617%
Valid Epoch: 1 	 Loss: 0.08140913024544716 	 Accuracy: 64.58333587646484%
model saves at 64.58333587646484 accuracy
-------------------------------------------------------------------------------
Train Epoch: 2 	 Loss: 0.08439010196399195 	 Accuracy: 63.73057174682617%
Valid Epoch: 2 	 Loss: 0.08189112320542336 	 Accuracy: 64.58333587646484%
-------------------------------------------------------------------------------
Train Epoch: 3 	 Loss: 0.08169856435894349 	 Accuracy: 63.73057174682617%
Valid Epoch: 3 	 Loss: 0.08216600120067596 	 Accuracy: 64.58